In [1]:
import pandas
from pandas.tools.plotting import scatter_matrix
import matplotlib.pyplot as plt
from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn import preprocessing
from sklearn.ensemble import IsolationForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
import pickle
from sklearn import metrics
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import ExtraTreesClassifier

def writeToFile(data, filename):
    with open(filename, 'w') as f:
        for item in data:
            f.write("%s\n" % item)
    f.close()
def printify(t):
    for x in t:
        print(x)
    

/opt/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Data reading from all chunks and making one  dataset

In [2]:
dataPrefix = '/home/data/aaia17/'
def readDataAsFrame(fNames):
    frame = []
    for fileName in fNames:
        fData = dataPrefix+fileName
        frame.append(pandas.read_csv(fData))
    return pandas.concat(frame)

In [3]:
trainingFiles = ['trainingData_tabular_chunk1.csv', 'trainingData_tabular_chunk2.csv',
                 'trainingData_tabular_chunk3.csv', 'trainingData_tabular_chunk4.csv']
rawTrainingDataset = readDataAsFrame(trainingFiles)

testFiles = ['testData_tabular.csv']
rawTestDataset = readDataAsFrame(testFiles)

In [4]:
print(rawTrainingDataset.groupby('decision').size())

decision
0     990401
1    1009599
dtype: int64


In [8]:
print(rawTrainingDataset.groupby('decision').size())

decision
0     990401
1    1009599
dtype: int64


In [84]:
print(rawTrainingDataset.shape)
print(rawTestDataset.shape)

(2000000, 45)
(750000, 45)


In [17]:
XRawTestDataset = rawTestDataset.values[:,2:45]
print(XRawTestDataset.shape)

(750000, 43)


#  Outliers detection and removing

In [4]:
YRawTrainingDataset = rawTrainingDataset.values[:,1]
XRawTrainingDataset = rawTrainingDataset.values[:,2:45]
XRawTestDataset = rawTestDataset.values[:,2:45]

In [86]:
%%time
YRawTrainingDataset = rawTrainingDataset.values[:,1]
XRawTrainingDataset = rawTrainingDataset.values[:,2:45]

rng = np.random.RandomState(42)
clf = IsolationForest(max_samples=10000, random_state=rng, n_jobs=-1)
clf.fit(XRawTrainingDataset)

trainingOutliers = clf.predict(XRawTrainingDataset)

CPU times: user 8min 5s, sys: 1min 4s, total: 9min 10s
Wall time: 9min 12s


In [87]:
XTrainigDatasetWithoutOutliers = XRawTrainingDataset[trainingOutliers>0]
YTrainingDatasetWithoutOutliers = YRawTrainingDataset[trainingOutliers>0]

In [5]:
print(XTrainigDatasetWithoutOutliers.shape)
print(XRawTrainingDataset.shape)

NameError: name 'XTrainigDatasetWithoutOutliers' is not defined

# Data scaling

In [5]:
def dataScaling(data, scaler):
    return scaler.fit_transform(data)

In [6]:
min_max_scaler = preprocessing.MinMaxScaler()

XTrainingDatasetScaledWithOutliers = dataScaling(XRawTrainingDataset, min_max_scaler)
XTestDatasetScaled = dataScaling(XRawTestDataset, min_max_scaler)

In [109]:
min_max_scaler = preprocessing.MinMaxScaler()

XTrainingDatasetScaled = dataScaling(XTrainigDatasetWithoutOutliers, min_max_scaler)
XTestDatasetScaled = dataScaling(XRawTestDataset, min_max_scaler)

In [16]:
print (XTrainingDatasetScaled.min(), XTestDatasetScaled.min(), 
    XTrainingDatasetScaled.max(), XTestDatasetScaled.max())

(0.0, 0.0, 1.0, 1.0)


In [35]:
XTestDatasetScaled.shape

(750000, 43)

In [21]:
Xtry = XTrainingDatasetScaled
Ytry = YTrainingDatasetWithoutOutliers
Xtry.shape

(1800000, 43)

# Feature selection 

In [22]:
%%time
estimator = LogisticRegression(random_state=rng)
selector = RFECV(estimator, step=1, cv=10, scoring='roc_auc', n_jobs=-1)
XTrainingSelected = selector.fit_transform(Xtry, Ytry)
pickle.dump(XTrainingSelected, open(dataPrefix+'XTrainingSelected.store',"wb"))

CPU times: user 4min 35s, sys: 14.9 s, total: 4min 50s
Wall time: 52min


In [25]:
XTrainingSelected.shape
selector.ranking_

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

## Grid Search

In [25]:
params_space = dict(svc__C=10.0**np.arange(-5,1),
                    svc__kernel=['rbf', 'linear', 'poly', 'sigmoid'],
                    svc__degree=np.arange(3,6),
                    svc__gamma=np.logspace(-9, 3, 4),
                    svc__probability=[True, False],
                    svc__shrinking=[True, False])

In [ ]:
grid = GridSearchCV(estimator, params_space, cv=10, scoring='roc_auc')

# Creating train and validation probes

In [110]:
validation_size = 0.30
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(XTrainingDatasetScaled, 
        YTrainingDatasetWithoutOutliers, test_size=validation_size, random_state=seed)

In [14]:
validation_size = 0.30
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(XTrainingDatasetScaledWithOutliers, 
        YRawTrainingDataset, test_size=validation_size, random_state=seed)

In [111]:
print(X_train.shape, X_validation.shape, Y_train.shape, Y_validation.shape)

((1260000, 43), (540000, 43), (1260000,), (540000,))


# Model collection

In [26]:
# Test options and evaluation metric
seed = 7
scoring = 'roc_auc'
models = []
models.append(('RFC', RandomForestClassifier(n_jobs=-1, n_estimators=40)))


# Models training

In [27]:
# evaluate each model in turn
%%time
results = []
names = []
for name, model in models:
    kfold = model_selection.KFold(n_splits=10, random_state=seed)
    cv_results = model_selection.cross_val_score(model, X_train, Y_train, cv=kfold, scoring=scoring)
    results.append(cv_results)
    names.append(name)
    msg = "%s: %f (%f)" % (name, cv_results.mean(), cv_results.std())
    print(msg)

RFC: 0.862889 (0.001159)


# Model training

In [30]:
X_train.shape
svc_model = svm.SVC(gamma=0.001, C=100., kernel='linear')
clf = svc_model.fit(X_train, Y_train)

(1440000, 43)

In [9]:
%%time
clf = RandomForestClassifier(n_jobs=-1, n_estimators=2000)
clf = clf.fit(XTrainingDatasetScaledWithOutliers, YRawTrainingDataset)

CPU times: user 15h 6min 56s, sys: 2min 31s, total: 15h 9min 28s
Wall time: 14min 27s


In [ ]:
%%time
clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=1500, class_weight="balanced", bootstrap=True)
clf = clf.fit(XTrainingDatasetScaledWithOutliers, YRawTrainingDataset)

In [12]:
print "Features sorted by their score:"
names = rawTrainingDataset.columns.values[0:43]
ranking = sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), range(0, 43)), reverse=True)
printify(ranking[0:32])

Features sorted by their score:
(0.0646, 4)
(0.0587, 16)
(0.0515, 41)
(0.0482, 42)
(0.0452, 40)
(0.0418, 31)
(0.0411, 3)
(0.0404, 34)
(0.0377, 26)
(0.0352, 29)
(0.0344, 15)
(0.0325, 33)
(0.0323, 32)
(0.031, 11)
(0.03, 27)
(0.0291, 28)
(0.0267, 9)
(0.026, 8)
(0.0257, 21)
(0.0219, 20)
(0.0212, 39)
(0.0197, 0)
(0.0173, 35)
(0.0161, 24)
(0.0157, 36)
(0.0152, 30)
(0.0143, 38)
(0.0143, 23)
(0.0142, 12)
(0.014, 25)
(0.013, 7)
(0.0117, 19)


In [13]:
topFeatures = [x[1] for x in ranking[0:32]]
topFeatures[2]

41

In [60]:
topFeatures

[6,
 18,
 43,
 44,
 42,
 5,
 33,
 28,
 36,
 34,
 31,
 17,
 35,
 13,
 29,
 30,
 11,
 10,
 23,
 22,
 41,
 2]

# Selected Feature Observations

In [14]:
def getNewForIndexes(data, indexes):
    X_selectedFeature = np.empty((len(data), 0), float)
    for index in indexes:
        X_selectedFeature = np.column_stack([X_selectedFeature, data[:,index]])
    return X_selectedFeature

In [15]:
X_TrainSelectedFeature = getNewForIndexes(XTrainingDatasetScaledWithOutliers, topFeatures)
X_TestSelectedFeature = getNewForIndexes(XTestDatasetScaled, topFeatures)

In [17]:
X_TestSelectedFeature.shape

(750000, 32)

In [18]:
%%time
clf = RandomForestClassifier(n_jobs=-1, n_estimators=1000)
clf = clf.fit(X_TrainSelectedFeature, YRawTrainingDataset)
predictedTestWon = clf.predict_proba(X_TestSelectedFeature)[:,1]
writeToFile(predictedTestWon, dataPrefix+'predicted8.txt')

CPU times: user 5h 17min 30s, sys: 1min 40s, total: 5h 19min 10s
Wall time: 5min 14s


In [144]:
print "Features sorted by their score:"
names = rawTrainingDataset.columns.values[0:43]
ranking = sorted(zip(map(lambda x: round(x, 4), clf.feature_importances_), range(0, 22)), reverse=True)
printify(ranking[0:22])

Features sorted by their score:
(0.0749, 0)
(0.0696, 1)
(0.0665, 2)
(0.0605, 3)
(0.0572, 4)
(0.0516, 6)
(0.0506, 5)
(0.0488, 8)
(0.0484, 7)
(0.0462, 11)
(0.0416, 10)
(0.0404, 9)
(0.0401, 12)
(0.0384, 13)
(0.0366, 14)
(0.036, 15)
(0.0357, 16)
(0.0356, 18)
(0.0334, 17)
(0.0294, 20)
(0.0294, 19)
(0.0292, 21)


# Prediction on Test Set

In [118]:
clf.classes_
predicted.shape

(600000,)

In [56]:
predictedTestWon.shape

(750000,)

In [28]:
predicted = clf.predict(X_validation)
predictedTestWon = predicted[:,1]
print(metrics.classification_report(Y_validation, predicted))

IndexError: too many indices for array

In [8]:
predictedTestWon = clf.predict_proba(XTestDatasetScaled)[:,1]
writeToFile(predictedTestWon, dataPrefix+'predicted9.txt')

In [24]:
models.append(('KNN', KNeighborsClassifier(n_jobs=60)))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC()))
models.append(('RF', RandomForestClassifier(n_jobs=60)))

array([  1.00000000e-09,   1.00000000e-05,   1.00000000e-01,
         1.00000000e+03])

In [7]:
clf = RandomForestClassifier(n_jobs=1, n_estimators=20)
# Utility function to report best scores
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
param_dist = {"max_depth": [3, None],
              "max_features": sp_randint(1, 11),
              "min_samples_split": sp_randint(1, 11),
              "min_samples_leaf": sp_randint(1, 11),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}     
n_iter_search = 20


In [71]:
random_search = RandomizedSearchCV(clf, param_distributions=param_dist, n_iter=n_iter_search)

(1440000,)

In [8]:
from time import time
from sklearn.externals import joblib
# use a full grid over all parameters
param_grid = {"max_depth": [3, None],
              "max_features": [5, 10, 20, 30],
              "min_samples_split": [2, 3, 10],
              "min_samples_leaf": [1, 3, 10],
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}

# run grid search
clf = RandomForestClassifier(n_jobs=-1, n_estimators=20)

grid_search = GridSearchCV(clf, param_grid=param_grid)
start = time()
grid_search.fit(XTrainingDatasetScaledWithOutliers, YRawTrainingDataset)

print("GridSearchCV took %.2f seconds for %d candidate parameter settings."
      % (time() - start, len(grid_search.cv_results_['params'])))
report(grid_search.cv_results_)



AttributeError: 'GridSearchCV' object has no attribute 'cv_results_'

In [10]:
grid_search.best_params_

{'bootstrap': False,
 'criterion': 'entropy',
 'max_depth': None,
 'max_features': 30,
 'min_samples_leaf': 1,
 'min_samples_split': 2}

In [12]:
clf = RandomForestClassifier(n_jobs=-1, n_estimators=1500, criterion="entropy", bootstrap=False, max_features=30)
clf = clf.fit(XTrainingDatasetScaledWithOutliers, YRawTrainingDataset)
predictedTestWon = clf.predict_proba(XTestDatasetScaled)[:,1]
writeToFile(predictedTestWon, dataPrefix+'predicted14.txt')

In [13]:
clf

RandomForestClassifier(bootstrap=False, class_weight=None,
            criterion='entropy', max_depth=None, max_features=30,
            max_leaf_nodes=None, min_impurity_split=1e-07,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=1500, n_jobs=-1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)